In [1]:
cwd = "/home/zzz0054/chen_data"
reads = ["C0RPF1aux_S3_R1_001"]
subF = "Alonso"

In [2]:
!mkdir -p {cwd}/trimmed_out/
!mkdir -p {cwd}/before_trim/
!mkdir -p {cwd}/quality_filtered
!mkdir -p {cwd}/after_trim/
!mkdir -p  {cwd}/aligned

In [3]:
!mkdir -p {cwd}/trimmed_out/
for read in reads:
    # -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -e 0.2 -m 80 -M 180
    !cutadapt -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -e 0.2 -m 15 -M 60 -o {cwd}/trimmed_out/{read}.trimmed.fastq {cwd}/{subF}/{read}.fastq > {cwd}/trimmed_out/{read}_trimmed.log

Done           00:04:44    26,599,133 reads @  10.7 µs/read;   5.61 M reads/minute


In [ ]:
# quality check
!mkdir {cwd}/before_trim/
for read in reads:
    !fastqc {cwd}/{subF}/{read}.fastq -o {cwd}/before_trim/ -a {cwd}/adapter.txt

mkdir: cannot create directory ‘/home/zzz0054/chen_data/before_trim/’: File exists
null
Started analysis of C0RPF1aux_S3_R1_001.fastq
Approx 5% complete for C0RPF1aux_S3_R1_001.fastq
Approx 10% complete for C0RPF1aux_S3_R1_001.fastq
Approx 15% complete for C0RPF1aux_S3_R1_001.fastq
Approx 20% complete for C0RPF1aux_S3_R1_001.fastq
Approx 25% complete for C0RPF1aux_S3_R1_001.fastq
Approx 30% complete for C0RPF1aux_S3_R1_001.fastq
Approx 35% complete for C0RPF1aux_S3_R1_001.fastq
Approx 40% complete for C0RPF1aux_S3_R1_001.fastq
Approx 45% complete for C0RPF1aux_S3_R1_001.fastq
Approx 50% complete for C0RPF1aux_S3_R1_001.fastq
Approx 55% complete for C0RPF1aux_S3_R1_001.fastq
Approx 60% complete for C0RPF1aux_S3_R1_001.fastq
Approx 65% complete for C0RPF1aux_S3_R1_001.fastq
Approx 70% complete for C0RPF1aux_S3_R1_001.fastq
Approx 75% complete for C0RPF1aux_S3_R1_001.fastq
Approx 80% complete for C0RPF1aux_S3_R1_001.fastq
Approx 85% complete for C0RPF1aux_S3_R1_001.fastq
Approx 90% comple

In [ ]:
!mkdir -p {cwd}/quality_filtered
for read in reads:
    !fastq_quality_filter -Q33 -v -q 25 -p 75 -i {cwd}/trimmed_out/{read}.trimmed.fastq -o {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq > {cwd}/quality_filtered/{read}.Qfilter.log

In [ ]:
# quality check
!mkdir {cwd}/after_trim/
for read in reads:
    !fastqc {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq -o {cwd}/after_trim/ -a {cwd}/adapter.txt

mkdir: cannot create directory ‘/home/zzz0054/chen_data/after_trim/’: File exists
null
Started analysis of C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 5% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 10% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 15% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 20% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 25% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 30% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 35% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 40% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 45% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 50% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 55% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 60% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.fastq
Approx 65% complete for C0RPF1aux_S3_R1_001.trimmed.Qfilter.

In [ ]:
!mkdir -p  {cwd}/aligned
for read in reads:
    !bowtie -n 2 -y -a --norc --best --strata -S -p 16 -l 25 --un={cwd}/aligned/noncontam_{read}.fastq {cwd}/bowtie_index/bowtie_index -q {cwd}/quality_filtered/{read}.trimmed.Qfilter.fastq {cwd}/aligned/{read}.alin

Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 546253
# reads with at least one alignment: 7321 (1.34%)
# reads that failed to align: 538932 (98.66%)
Reported 12709 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 3181526
# reads with at least one alignment: 71327 (2.24%)
# reads that failed to align: 3110199 (97.76%)
Reported 133214 alignments
Setting the index via positional argument will be deprecated in a future release. Please use -x option instead.
# reads processed: 505132
# reads with at least one alignment: 7744 (1.53%)
# reads that failed to align: 497388 (98.47%)
Reported 13986 alignments


In [ ]:
for read in reads:
    !STAR --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir {cwd}/STAR_genome_index/ --readFilesIn {cwd}/aligned/noncontam_{read}.fastq --outFileNamePrefix  {cwd}/star_out/{read}/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All

	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 --outFilterType Normal --outWigType wiggle --outWigStrand Stranded --outWigNorm RPM --alignEndsType EndToEnd --outFilterMismatchNmax 1 --outFilterMultimapNmax 1 --genomeDir /home/zzz0054/chen_data/STAR_genome_index/ --readFilesIn /home/zzz0054/chen_data/aligned/noncontam_C0RPF1aux_S3_R1_001.fastq --outFileNamePrefix /home/zzz0054/chen_data/star_out/C0RPF1aux_S3_R1_001/ --outSAMtype BAM SortedByCoordinate --quantMode TranscriptomeSAM GeneCounts --outSAMattributes All
	STAR version: 2.7.11b   compiled: 2024-03-19T08:38:59+0000 :/opt/conda/conda-bld/star_1710837244939/work/source
Jul 22 10:30:20 ..... started STAR run
Jul 22 10:30:20 ..... loading genome
Jul 22 10:30:23 ..... started mapping
Jul 22 10:32:26 ..... finished mapping
Jul 22 10:32:27 ..... started sorting BAM
Jul 22 10:32:27 ..... started wiggle output
Jul 22 10:32:31 ..... finished successfully
	/home/zzz0054/miniconda3/envs/ribo4/bin/STAR-avx2 --runThreadN 8 -

In [ ]:
for read in reads:
    !samtools sort -T {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted -o {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam {cwd}/star_out/{read}/Aligned.toTranscriptome.out.bam
    !samtools index {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam ## mapped to transcriptome
    !samtools index {cwd}/star_out/{read}/Aligned.sortedByCoord.out.bam ## mapped to genome

In [ ]:
for read in reads:
    !mkdir -p {cwd}/metaplots/{read}/
    !metaplots -a {cwd}/ref/annot -r {cwd}/star_out/{read}/Aligned.toTranscriptome.out.sorted.bam -o {cwd}/metaplots/{read}/

[2024-07-22 10:39:54] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
No obviously periodicity are detected from bam file /home/zzz0054/chen_data/star_out/C0RPF1aux_S3_R1_001/Aligned.toTranscriptome.out.sorted.bam,
it could be due to poor quality sequencing.
Please check the metagene plots and try again by lowering the value of frame0_percent
[2024-07-22 10:39:57] Complete prediction of the P-site locations
[2024-07-22 10:40:00] Create metaplot file and predict the P-site locations ...
	Loading transcripts.pickle ...
No obviously periodicity are detected from bam file /home/zzz0054/chen_data/star_out/C0RPF2aux_S4_R1_001/Aligned.toTranscriptome.out.sorted.bam,
it could be due to poor quality sequencing.
Please check the metagene plots and try again by lowering the value of frame0_percent
[2024-07-22 10:40:04] Complete prediction of the P-site locations
[2024-07-22 10:40:07] Create metaplot file and predict the P-site locations ...
	Loading trans